
# 1) Configuration de PRAW (Python Reddit Api Wrapper)
## !! A faire avant de pouvoir utiliser n'importe laquelle des parties suivantes !!

In [1]:
import praw               # API pour reddit
import pandas as pd       # pour afficher les dictionnaires sous forme de tableaux
import datetime as dt     # Pour convertir la date au bon format
import os
import sys
import shutil

In [2]:
# Créer la variable de configuration pour accéder aux requêtes vers reddit
reddit = praw.Reddit(client_id='BEjar6X3GYV5Vw', \
                     client_secret='lg6D6DXG14FH4kHtGVt7cins5OY', \
                     user_agent='feelcrypt', \
                     username='feelcrypt', \
                     password='spiderminute38')

# 2) Générer la liste des posts à traiter
## !! Ne lancer qu'une fois par subreddit !!

Cette partie va récupérer les 500 top posts du subreddit défini et sauvegarder uniquement les id de ces posts dans une liste de post à traiter (un fichier txt)

## 2.1) Variables globales

In [3]:
# Variables de configuration globale
nb_top_posts = 5  # nombre de posts selectionnés parmi les premiers (limite = 500)
subreddit_title = 'btc' # Définir le titre du subreddit que l'on va cibler

## 2.2) Fonctions

In [4]:
def get_script_dir_path():
    script_dir_path = %pwd 
    script_dir_path += '\\'
    return script_dir_path

In [5]:
# Vérifier si le dossier passé en paramètre (+chemin relatif au dossier du subreddit) existe, sinon le créer
def create_folder(subreddit_path, folder_name):
    folder_name = folder_name
    folder_path = subreddit_path + folder_name + '\\'
    try:
        os.makedirs(folder_path)
        print('Dossier créé : ' + folder_name) # Log
    except FileExistsError:
        print('Dossier déjà existant : ' + folder_name) # Log
        pass
    return folder_path

In [6]:
# Créer la liste des id des posts à traiter
def create_id_todo(subreddit_path):
    # Créer la variable sur le subreddit
    subreddit = reddit.subreddit(subreddit_title)
    
    # Récupérer les n premiers posts de notre subreddit de notre liste
    top_posts = []

    for post in subreddit.top(limit = nb_top_posts):
        top_posts.append(post)

    # Enregistrer les id dans une liste
    top_posts_id = []
    for post in top_posts:
        top_posts_id.append(post.id)

    # Enregistrer la liste des id dans un fichier txt

    # Créer le chemin de la liste d'id à traiter
    file_name = 'id_todo.txt'
    id_todo__path = subreddit_path + file_name

    # Remplir la liste avec les id
    top_posts_id_file = open(id_todo__path,'w')
    for post_id in top_posts_id:
         top_posts_id_file.write(post_id)
         top_posts_id_file.write('\n')
    top_posts_id_file.close()
    
    # Log
    print('Fichier initialisé : ' + file_name)
    
    return id_todo__path

In [7]:
# Créer le fichier pour compter le nombre de commentaires total
# Le créer et le réinitialiser à 0 s'il existe déjà
def create_counter(subreddit_path):
    file_name = 'comments_counter.txt'
    counter__path = subreddit_path + file_name
    nb_file = open(counter__path,'w').close()
    nb_file = open(counter__path,'w')
    nb_file.write(str(0))
    nb_file.close()
    
    # log
    print('Fichier initialisé : ' + file_name)
    
    return counter__path

## 2.3) Script

In [9]:
# Récupérer le chemin du dossier contenant le script python
script_folder = get_script_dir_path()

# Créer un dossier pour le subreddit
subreddit_path = create_folder(script_folder, subreddit_title)

# Vérifier si le dossier comments (chemin relatif) existe, sinon le créer
comments_path = create_folder(subreddit_path, 'comments')

# Vérifier si le dossier comments (chemin relatif) existe, sinon le créer
comments_manager_path = create_folder(subreddit_path, 'comments_manager')

# Créer la liste des id des posts à traiter
id_todo_path = create_id_todo(subreddit_path)

# Créer le fichier pour compter le nombre de commentaires total
counter_path = create_counter(subreddit_path)

Dossier déjà existant : btc
Dossier déjà existant : comments
Dossier créé : comments_manager
Fichier initialisé : id_todo.txt
Fichier initialisé : comments_counter.txt


# 3) Traiter les posts

## 3.1) Variables globales

In [9]:
# Variables globales

scrap_all_comments = False
# Si false : prend seulement les 32 premiers commentaires de chaque post
# Si true : prend tous les commentaires du post

scrap_responses_to_comments = False
# Si false : ne prend que les réponses directes au post
# Si true : prend également en compte les réponses aux commentaires

## 3.2) Functions

In [10]:
# Fonction de conversion pour la date avec timestamp
def get_date(created):
    return dt.date.fromtimestamp(created)

In [11]:
# Ajouter le nb de commentaires traités dans le compteur
def updating_counter(counter_path, nb_comments, current_post_id):
        nb_file = open(counter_path,'r')
        total_comments = int(nb_file.read())
        total_comments += nb_comments
        nb_file.close()
        nb_file = open(counter_path,'w').close()
        nb_file = open(counter_path,'w')
        nb_file.write(str(total_comments))
        nb_file.close()
        
        print(str(nb_comments) + " comments from post " + str(current_post_id))
        print('total comments saved = ' + str(total_comments))

In [12]:
# Traiter chaque id du fichier todo
def get_comments(id_todo):
    while id_todo:  # Vérifier si la liste n'est pas vide

        # Récupérer le post
        post = reddit.submission(id = id_todo[0])
        current_post_id = post.id
        print('-----------------------------------')
        print('Starting working on post : ' + str(current_post_id))

        ## Récupérer les commentaires du poste
        
        #Réinitialiser le dictionnaire
        comments_dict = { "created_utc":[], \
                         "body":[], \
                         "score":[], \
                         "nb_replies":[], \
                         "stickied":[], \
                         "author":[], \
                         "id":[], \
                         "post_title":[], \
                         "post_link":[]} 

        #se débarasser récursivement de la limite de 32 commentaires par requête
        if scrap_all_comments:
            post.comments.replace_more(limit=None)  # prendre en compte les commentaires supp
        else:
            post.comments.replace_more(limit=0)    # ignorer les commentaires supp

        # Récupérer la liste des commentaires
        if scrap_responses_to_comments:
            comments_list = post.comments.list()
        else:
            comments_list = post.comments

        # Remplir le dictionnaire de commentaires avec la liste  
        for comment in comments_list:
            comments_dict["created_utc"].append(comment.created_utc)
            comments_dict["body"].append(comment.body)
            comments_dict["score"].append(comment.score)
            comments_dict["nb_replies"].append(len(comment.replies))
            comments_dict["stickied"].append(comment.stickied)
            comments_dict["author"].append(comment.author)
            comments_dict["id"].append(comment.id)
            comments_dict["post_title"].append(comment.submission.title)
            comments_dict["post_link"].append(comment.submission.url)

        # mettre la data au format pandas (qui permet de faire un "tableur" à partir du dictionnaire)
        comments_data = pd.DataFrame(comments_dict)

        # Créer la liste des dates converties et la sauvgarder dans la variable _timstamp
        # created_utc est la colonne contenant les dates au mauvais format
        _timestamp = comments_data["created_utc"].apply(get_date)

        # ajouter la liste à une nouvelle colonne appelée timestamp
        comments_data = comments_data.assign(date = _timestamp)

        # Supprimer la colonne du temps inutile
        comments_data = comments_data.drop(columns="created_utc")

        # Déplacer la date en première position
        colonnes = comments_data.columns.tolist()
        colonnes = colonnes[-1:] + colonnes[:-1]
        comments_data = comments_data[colonnes]

        # Trier par la colonne date
        comments_sorted = comments_data.sort_values(by=['date'])

        ## Extraire plusieurs dataframe qui représentent chacun une date avec tous les commentaires dedans
        # Récupérer la liste des dates et indexer par dates
        comments_sorted.set_index(keys=['date'], drop=False,inplace=True)
        dates = comments_sorted['date'].unique().tolist()

        # Enregistrer dans une liste contenant chaque dataframe (1 data frame = 1 date)
        comments_splitperday = []
        for date in dates:
            comments_per_day = pd.DataFrame(comments_sorted.loc[comments_sorted.date == date])
            comments_splitperday.append(comments_per_day)

        # Enregistrer chaque dataframe dans un fichier csv
        for dataframe in comments_splitperday:
            # Récupérer la date du dataframe supprimer la colonne date
            date = str(dataframe.date.iloc[0])
            dataframe = dataframe.drop(columns="date")
            csv_path = comments_path + "\\" + date + '.csv'

            # Enregistrer au format csv avec pour nom la date
            dataframe.to_csv(csv_path, ';', mode='a', index=False, header=True) 

        # Retirer le premier id de la liste, le laisser à la fin, si le while est interrompu, il sera retiré alors que le post n'aura pas été traité
        id_todo.pop(0)

        # Ajouter l'id du post lu dans la liste des id_done
        id_done_path = subreddit_path + "id_done.txt"
        with open(id_done_path, "a") as file:
            file.write(current_post_id)

        # Enregistrer la nouvelle liste des id_todo dans le fichier txt (ou le créer s'il n'existe pas encore)
        with open(id_todo_path, "r") as file:
            data = file.read()
        with open(id_todo_path, "w") as file:
            for post_id in id_todo:
                file.write(post_id)

        # Ajouter le nb de commentaires traités dans le compteur
        updating_counter(counter_path, len(comments_data.index), current_post_id)
        
    else:
        print('============================================')
        print("La liste est vide, tout a été traité")

## 3.3) Script

In [13]:
# Ouvrir la liste de posts pas encore faits
with open(id_todo_path) as file:
  id_todo = file.readlines()

In [14]:
# Traiter chaque id du fichier todo
get_comments(id_todo)

-----------------------------------
Starting working on post : 7eil12

1193 comments from post 7eil12

total comments saved = 1193
-----------------------------------
Starting working on post : 7opi7w

762 comments from post 7opi7w

total comments saved = 1955
-----------------------------------
Starting working on post : 7hzklb

1216 comments from post 7hzklb

total comments saved = 3171
-----------------------------------
Starting working on post : 7kxfor

1020 comments from post 7kxfor

total comments saved = 4191
-----------------------------------
Starting working on post : 7dzqhm

504 comments from post 7dzqhm

total comments saved = 4695
-----------------------------------
Starting working on post : 7ih0hd

637 comments from post 7ih0hd

total comments saved = 5332
-----------------------------------
Starting working on post : 6gek0z

477 comments from post 6gek0z

total comments saved = 5809
-----------------------------------
Starting working on post : 7i7qch

150 comments fro

281 comments from post 7g9dd5

total comments saved = 26464
-----------------------------------
Starting working on post : 7nm630

113 comments from post 7nm630

total comments saved = 26577
-----------------------------------
Starting working on post : 7wnf9c

188 comments from post 7wnf9c

total comments saved = 26765
-----------------------------------
Starting working on post : 7o73vr

420 comments from post 7o73vr

total comments saved = 27185
-----------------------------------
Starting working on post : 7lvie2

316 comments from post 7lvie2

total comments saved = 27501
-----------------------------------
Starting working on post : 7gg5qt

165 comments from post 7gg5qt

total comments saved = 27666
-----------------------------------
Starting working on post : 94oh8t

42 comments from post 94oh8t

total comments saved = 27708
-----------------------------------
Starting working on post : 8rz0uy

343 comments from post 8rz0uy

total comments saved = 28051
------------------------

214 comments from post 7sgtj9

total comments saved = 41381
-----------------------------------
Starting working on post : 8arp51

306 comments from post 8arp51

total comments saved = 41687
-----------------------------------
Starting working on post : 7a9uha

351 comments from post 7a9uha

total comments saved = 42038
-----------------------------------
Starting working on post : 8i6ugd

161 comments from post 8i6ugd

total comments saved = 42199
-----------------------------------
Starting working on post : 78vkdu

180 comments from post 78vkdu

total comments saved = 42379
-----------------------------------
Starting working on post : 6r3zko

250 comments from post 6r3zko

total comments saved = 42629
-----------------------------------
Starting working on post : 7n7iq6

188 comments from post 7n7iq6

total comments saved = 42817
-----------------------------------
Starting working on post : 7ge27h

426 comments from post 7ge27h

total comments saved = 43243
-----------------------

276 comments from post 7n15mn

total comments saved = 54924
-----------------------------------
Starting working on post : 7foqr3

87 comments from post 7foqr3

total comments saved = 55011
-----------------------------------
Starting working on post : 83iruo

22 comments from post 83iruo

total comments saved = 55033
-----------------------------------
Starting working on post : att3zw

53 comments from post att3zw

total comments saved = 55086
-----------------------------------
Starting working on post : 8612wg

268 comments from post 8612wg

total comments saved = 55354
-----------------------------------
Starting working on post : 7nxo9j

151 comments from post 7nxo9j

total comments saved = 55505
-----------------------------------
Starting working on post : abteqs

167 comments from post abteqs

total comments saved = 55672
-----------------------------------
Starting working on post : 7rfog0

497 comments from post 7rfog0

total comments saved = 56169
--------------------------

95 comments from post 7d5cky

total comments saved = 68089
-----------------------------------
Starting working on post : 76ge7b

92 comments from post 76ge7b

total comments saved = 68181
-----------------------------------
Starting working on post : 6wy0ss

275 comments from post 6wy0ss

total comments saved = 68456
-----------------------------------
Starting working on post : 8w1rew

123 comments from post 8w1rew

total comments saved = 68579
-----------------------------------
Starting working on post : 8ef1hv

112 comments from post 8ef1hv

total comments saved = 68691
-----------------------------------
Starting working on post : 7r1qvs

49 comments from post 7r1qvs

total comments saved = 68740
-----------------------------------
Starting working on post : 7hmgym

165 comments from post 7hmgym

total comments saved = 68905
-----------------------------------
Starting working on post : 7kyrrk

255 comments from post 7kyrrk

total comments saved = 69160
--------------------------

544 comments from post 6jqrub

total comments saved = 81497
-----------------------------------
Starting working on post : 7l5mra

96 comments from post 7l5mra

total comments saved = 81593
-----------------------------------
Starting working on post : 9h6rch

168 comments from post 9h6rch

total comments saved = 81761
-----------------------------------
Starting working on post : 7d252y

410 comments from post 7d252y

total comments saved = 82171
-----------------------------------
Starting working on post : 9niuo8

475 comments from post 9niuo8

total comments saved = 82646
-----------------------------------
Starting working on post : 75vksk

413 comments from post 75vksk

total comments saved = 83059
-----------------------------------
Starting working on post : 90ghws

99 comments from post 90ghws

total comments saved = 83158
-----------------------------------
Starting working on post : 6rzwhy

161 comments from post 6rzwhy

total comments saved = 83319
-------------------------

321 comments from post 6qftjc

total comments saved = 93533
-----------------------------------
Starting working on post : 7j0weq

221 comments from post 7j0weq

total comments saved = 93754
-----------------------------------
Starting working on post : 7y1rso

52 comments from post 7y1rso

total comments saved = 93806
-----------------------------------
Starting working on post : 7jahjz

41 comments from post 7jahjz

total comments saved = 93847
-----------------------------------
Starting working on post : 7g1b0f

332 comments from post 7g1b0f

total comments saved = 94179
-----------------------------------
Starting working on post : 7d94is

414 comments from post 7d94is

total comments saved = 94593
-----------------------------------
Starting working on post : 6uo8oo

141 comments from post 6uo8oo

total comments saved = 94734
-----------------------------------
Starting working on post : 8wjr3x

183 comments from post 8wjr3x

total comments saved = 94917
-------------------------

265 comments from post 8qsu3l

total comments saved = 104182
-----------------------------------
Starting working on post : 7u7yij

56 comments from post 7u7yij

total comments saved = 104238
-----------------------------------
Starting working on post : 7p7evu

166 comments from post 7p7evu

total comments saved = 104404
-----------------------------------
Starting working on post : 7ddzmr

240 comments from post 7ddzmr

total comments saved = 104644
-----------------------------------
Starting working on post : 6wc9he

65 comments from post 6wc9he

total comments saved = 104709
-----------------------------------
Starting working on post : 7jivvf

171 comments from post 7jivvf

total comments saved = 104880
-----------------------------------
Starting working on post : 7ei4du

78 comments from post 7ei4du

total comments saved = 104958
-----------------------------------
Starting working on post : 7h0isc

131 comments from post 7h0isc

total comments saved = 105089
------------------

# 4) Supprimer toute la data
utile uniquement en debbug, permet de supprimer tous les dossiers générés et leur contenu

In [11]:
def Delete_all_data():
    if os.path.isdir(subreddit_path):
        shutil.rmtree(subreddit_path)
        print('Dossier supprimé : ' + subreddit_path)
    else:
        print('Dossier inexistant : ' + subreddit_path)

In [36]:
Delete_all_data()

Dossier supprimé : C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V3\btc\
